In [1]:
import sage.all
from sage.calculus.var import var, function
from sage.misc.reset import reset
from sage.calculus.functional import diff


from sage.symbolic.relation import solve

from delierium.Infinitesimals import prolongationODE
from delierium.JanetBasis import Janet_Basis
y = function('y')
x = var('x')
#ode=4*diff(y(x),x,x)*y(x) - 3* diff(y(x),x)**2-12*y(x)**3
#p=prolongationODE(ode, y, x)[0].expand()
#p.show()
#s1=solve(ode==0, diff(y(x),x,x))
#s1=s1[0];s1.show();s1.lhs().show();s1.rhs().show()
#ode1=p.subs({s1.lhs() : s1.rhs()}).simplify()
#ode1.expand().show()
#r1=ode1.expand().collect(diff(y(x),x)**3);r1.show()
#r2=r1.collect(diff(y(x),x)**2);r2.show()
#r3=r2.collect(diff(y(x),x)); r3.show()
#r4=r3.collect(y(x)); r4.show()
from IPython.core.debugger import set_trace
function('xi phi')
def infinitesimalsODE (ode, dependent, independent):
    def order(ode, dep, indep):
        return 3
    prolongation = prolongationODE(ode, dependent, independent)[0]
    prolongation.show()
    l = []
    def collect(prol, d):
        res = 0
        s = prol.operands()
        for _s in s:
            locs = _s.operands()
            if d in locs:
                res += _s/d
        return res
    for o in range (order(ode, y, x),0,-1):
        d = diff(dependent(independent), independent)**o
        c = collect(prolongation, d)
        l.append(c.expand())
        prolongation  = prolongation - c*d
    l.append (prolongation.expand())
    print (";;;;;;;;;;;;;;;;;;;;;")
    for _ in l:
        _.show()
    janet = Janet_Basis(l, [phi, xi], [y,x])
    return janet

ode = 4*diff(y(x),x,x)*y(x) - 3* diff(y(x),x)**2-12*y(x)**3
inf = infinitesimalsODE(ode, y ,x)
inf.show()


/home/tapir/mambaforge/envs/sage/lib/python3.9/site-packages/sage/symbolic/expression_conversions.py:2347: DeprecationWarning: Substitution using function-call syntax and unnamed arguments is deprecated and will be removed from a future release of Sage; you can use named arguments instead, like EXPR(x=..., y=...)
See http://trac.sagemath.org/5930 for details.
  return self.new(*[self(_) for _ in ex.operands()])
/home/tapir/mambaforge/envs/sage/lib/python3.9/site-packages/sage/symbolic/operators.py:121: DeprecationWarning: Substitution using function-call syntax and unnamed arguments is deprecated and will be removed from a future release of Sage; you can use named arguments instead, like EXPR(x=..., y=...)
See http://trac.sagemath.org/5930 for details.
  return self._f(*args).diff(*vars)


-4*y(x)*D[0, 0](xi)(y(x), x)*diff(y(x), x)^3 + 36*xi(y(x), x)*y(x)^2*diff(y(x), x) + 4*y(x)*D[0, 0](phi)(y(x), x)*diff(y(x), x)^2 - 8*y(x)*D[0, 1](xi)(y(x), x)*diff(y(x), x)^2 + 6*D[0](xi)(y(x), x)*diff(y(x), x)^3 - 12*y(x)*D[0](xi)(y(x), x)*diff(y(x), x)*diff(y(x), x, x) - 36*phi(y(x), x)*y(x)^2 + 8*y(x)*D[0, 1](phi)(y(x), x)*diff(y(x), x) - 4*y(x)*D[1, 1](xi)(y(x), x)*diff(y(x), x) - 6*D[0](phi)(y(x), x)*diff(y(x), x)^2 + 6*D[1](xi)(y(x), x)*diff(y(x), x)^2 + 4*y(x)*D[0](phi)(y(x), x)*diff(y(x), x, x) - 8*y(x)*D[1](xi)(y(x), x)*diff(y(x), x, x) + 2*xi(y(x), x)*diff(y(x), x)*diff(y(x), x, x) - 4*xi(y(x), x)*y(x)*diff(y(x), x, x, x) - 2*(18*y(x)^2*diff(y(x), x) + diff(y(x), x)*diff(y(x), x, x) - 2*y(x)*diff(y(x), x, x, x))*xi(y(x), x) + 4*y(x)*D[1, 1](phi)(y(x), x) - 6*D[1](phi)(y(x), x)*diff(y(x), x) + 4*phi(y(x), x)*diff(y(x), x, x)

;;;;;;;;;;;;;;;;;;;;;


-4*y(x)*D[0, 0](xi)(y(x), x) + 6*D[0](xi)(y(x), x)

4*y(x)*D[0, 0](phi)(y(x), x) - 8*y(x)*D[0, 1](xi)(y(x), x) - 6*D[0](phi)(y(x), x) + 6*D[1](xi)(y(x), x)

36*xi(y(x), x)*y(x)^2 - 12*y(x)*D[0](xi)(y(x), x)*diff(y(x), x, x) + 8*y(x)*D[0, 1](phi)(y(x), x) - 4*y(x)*D[1, 1](xi)(y(x), x) + 2*xi(y(x), x)*diff(y(x), x, x) - 6*D[1](phi)(y(x), x)

-36*xi(y(x), x)*y(x)^2*diff(y(x), x) - 36*phi(y(x), x)*y(x)^2 + 4*y(x)*D[0](phi)(y(x), x)*diff(y(x), x, x) - 8*y(x)*D[1](xi)(y(x), x)*diff(y(x), x, x) - 2*xi(y(x), x)*diff(y(x), x)*diff(y(x), x, x) + 4*y(x)*D[1, 1](phi)(y(x), x) + 4*phi(y(x), x)*diff(y(x), x, x)

D[0, 0](phi)(y(x), x) + (-2) * D[0, 1](xi)(y(x), x) + (-3/2) * D[0](phi)(y(x), x) + (3/2) * D[1](xi)(y(x), x)
D[0, 1](phi)(y(x), x) + (-1/2) * D[1, 1](xi)(y(x), x) + (-3/4) * D[1](phi)(y(x), x) + (-3/2) * D[0](xi)(y(x), x) + (9/2*y(x)^2 + 1/4) * xi(y(x), x)
D[1, 1](phi)(y(x), x) + D[0](phi)(y(x), x) + (-2) * D[1](xi)(y(x), x) + (-9*y(x)^2 + 1) * phi(y(x), x) + (-9*y(x)^2 - 1/2) * xi(y(x), x)
D[0, 0](xi)(y(x), x) + (-3/2) * D[0](xi)(y(x), x)


In [2]:
for _ in inf.S:
    _.show()

-3/2*D[0](phi)(y(x), x) + D[0, 0](phi)(y(x), x) - 2*D[0, 1](xi)(y(x), x) + 3/2*D[1](xi)(y(x), x)

1/4*(18*y(x)^2 + 1)*xi(y(x), x) + D[0, 1](phi)(y(x), x) - 3/4*D[1](phi)(y(x), x) - 3/2*D[0](xi)(y(x), x) - 1/2*D[1, 1](xi)(y(x), x)

-(9*y(x)^2 - 1)*phi(y(x), x) - 1/2*(18*y(x)^2 + 1)*xi(y(x), x) + D[0](phi)(y(x), x) + D[1, 1](phi)(y(x), x) - 2*D[1](xi)(y(x), x)

-3/2*D[0](xi)(y(x), x) + D[0, 0](xi)(y(x), x)